# Scalable Supervised Machine Learning on the ODC

This notebook is a placeholder for now.  It will contain a markdown only description of the scripts in this folder, including:
- The guiding principles/philosophy of the scripts (ie setting expectations around what they can and cannot do)
- The order to run the scripts in (including links to each script)
- Any prerequisites required to understand the content in these notebooks (e.g. reading material, ODC/Sandbox/notebooks/python training material)
- Information on any set-up that might be required to run these notebooks (e.g. creating folders for outputs? Setting up a S3 bucket for intermediate files?)


---

## TO DO
- add a `debug=True|False` flag to `collect_training_data`, this will set ncpus=1 and remove HiddenPrints()
- add WOfS loading option to `collect_training_data`?
- add a check to the `collect_training_data` function that ensures the classes are integers, currently it runs the whole script before crashing at the last step
- See if we can merge `predict_xr` with `predict_proba_xr` as its silly to have to load all the data twice... not sure how yet: maybe with two apply_ufuncs? Or, stack predict&proba together as arrays as per https://stackoverflow.com/questions/52094320/with-xarray-how-to-parallelize-1d-operations-on-a-multidimensional-dataset
- rewrite `predict_xr` in light of `dask-ml`

In [ ]:
def predict_xr(model, input_xr):
    with joblib.parallel_backend('dask'):
        
        input_data = []

        for var_name in input_xr.data_vars:
            input_data.append(input_xr[var_name])

        input_data_flattened = []
        for data in args:
            input_data_flattened.append(data.flatten())

        # Flatten array
        input_data_flattened = np.array(input_data_flattened).transpose()

        # Mask out no-data in input (not all classifiers can cope with
        # Inf or NaN values)
        input_data_flattened = np.where(np.isfinite(input_data_flattened),
                                        input_data_flattened, 0)

        # Actually apply the classification
        out_class = model.predict(input_data_flattened)

        # Mask out NaN or Inf values in results
        out_class = np.where(np.isfinite(out_class), out_class, 0)

        # Reshape when writing out
        out_class = out_class.reshape(args[0].shape)

        # Set the stacked coordinate to match the input
        output_xr = xr.DataArray(out_class, coords=input_xr.coords)
        
        return output_xr